In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time

In [6]:
import HandTrackingModule as htm
import os

In [3]:
pTime = 0
cTime = 0

### Storing the images

In [47]:
folderPath = 'Images'
myList = os.listdir(folderPath)
print(myList)

['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg']


### Creating a list of images

In [48]:
overlayList = []

for imgPath in myList:
    image  = cv2.imread(f'{folderPath}/{imgPath}')
    overlayList.append(cv2.resize(image,(150,150)))

In [49]:
overlayList[0].shape

(150, 150, 3)

### List of Indices of finger tips and references 

Since we have defined the fixed set of gestures , we will define the list of mediapipe hand detection indices pertaining to the tip of each finger

In [40]:
tip_ids = [4,8,12,16,20]
reference_ids = [3,7,11,15,19]

In [51]:
pTime = 0
cTime = 0
cap = cv2.VideoCapture(0)
detector = htm.HandDetector()
while True:
    success, img = cap.read() 
    img = detector.findHands(img)
    landmark_list = detector.findPosition(img)
    
    # For counting, we are interested in the tip of fingers and a reference
    # Reference can be any point on that finger below the tip
    if landmark_list:
        fingers = []

        # We are checking y coordinates of tip and reference but for thumb this may not be enough
        # For thumb we can either change the reference or check x coordinate of x and reference
        
        # Approach 1
        # for tip_id,reference_id in zip(tip_ids,reference_ids):
        #     if landmark_list[tip_id][2] > landmark_list[reference_id][2]:
        #         fingers.append(0)
        #     else:
        #         fingers.append(1)

        # Approach 2

        #Thumb finger
        if landmark_list[tip_ids[0]][1] < landmark_list[tip_ids[0]-1][1]:
            fingers.append(0)
        else:
            fingers.append(1)

        for i in range(1,len(tip_ids)):
            if landmark_list[tip_ids[i]][2] > landmark_list[tip_ids[i] - 2][2]:
                fingers.append(0)
            else:
                fingers.append(1)

        #print(fingers)
        #print(sum(fingers))

        # Overlaying the loaded images on the input
        h, w, c = overlayList[0].shape
        img[0:w,0:h] = overlayList[sum(fingers)]

        # Adding a visual aid to depict the number detected
        cv2.rectangle(img, (20, 225), (170, 425), (0, 255, 0), cv2.FILLED)
        cv2.putText(img, str(sum(fingers)), (45, 375), cv2.FONT_HERSHEY_PLAIN,
                    10, (255, 0, 0), 25)
        

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime =cTime
    cv2.putText(img,str(int(fps)),(518,78),cv2.FONT_HERSHEY_COMPLEX,3,(255,255,255),4)

    cv2.imshow('Video',img)
    #cv2.waitKey(1)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [32]:
cv2.destroyAllWindows()